In [1]:
import json
import pandas as pd

import portrisk

from deribit import get_order_book

pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [2]:
# Load stress parameters
file_path = './parameters/spot_vol_shocks.json'

with open(file_path, 'r') as f:
    spot_vol_shocks_param = json.load(f)

crypto_parameters = portrisk.core.utils.CryptoParameters(spot_vol_shocks_param)


In [64]:
# load position data
df = pd.read_excel('positions.xlsx')

# Splitting the 'instrument' column by "-"
df[['underlying','expiry','strike','put_call']] = df['instrument'].str.split('-', expand=True)
df['strike'] = df['strike'].astype(float)
df['put_call'] = df['put_call'].map({'P':'put','C':'call'})
df['expiry'] = pd.to_datetime(df['expiry'], format='%d%b%y')

# Calculate time to expiry
valuation_day = pd.to_datetime('2024-08-05')
df['time_to_expiry'] = (df['expiry'] - valuation_day).dt.days / 365

# get market data from deribit
deribit_res = get_order_book(instruments=df['instrument'].to_list())
df = pd.merge(df, deribit_res,how='left',left_on='instrument',right_on='instrument_name')


# market_price	spot		
df['cost_of_carry_rate'] = 'default'
df['multiplier'] = 1
df['rate'] = 0.03
df['vol'] = df['mark_iv'] / 100
df['spot'] = df['index_price'].values[0]


In [66]:
spot_vol_shocks = portrisk.crypto.CryptoSpotVolShocks(crypto_parameters)

In [78]:
post_stress, col = spot_vol_shocks.apply_spot_vol_shocks(df)

In [82]:
post_stress

,instrument,quantity,underlying,expiry,strike,put_call,time_to_expiry,underlying_price,timestamp,mark_iv,instrument_name,index_price,rho,theta,vega,gamma,delta,bid_iv,ask_iv,best_bid_price,best_ask_price,cost_of_carry_rate,multiplier,rate,vol,spot,spot_shock,vol_shock,scenario_1,scenario_2
0,BTC-25OCT24-50000-P,-25.0,BTC,2024-10-25,50000.0,put,0.232877,55711.1600,1722896481181,58.60,BTC-25OCT24-50000-P,54681.59,-43.95354,-33.01789,90.60740,0.00002,4.301911e+05,57.75,59.29,0.0590,0.0615,default,1,0.03,0.5860,54681.59,0.15,0,35216.595972,47835.761852
1,BTC-25OCT24-84000-C,0.0,BTC,2024-10-25,84000.0,call,0.232877,55711.3923,1722896482188,64.98,BTC-25OCT24-84000-C,54684.77,12.38743,-20.66699,51.14403,0.00001,0.000000e+00,64.09,65.19,0.0145,0.0155,default,1,0.03,0.6498,54681.59,0.15,0,0.000000,0.000000
2,BTC-27DEC24-105000-C,-8.8,BTC,2024-12-27,105000.0,call,0.405479,56644.7900,1722896483548,68.88,BTC-27DEC24-105000-C,54684.77,20.60348,-16.28672,67.81809,0.00001,-5.168133e+04,68.15,69.41,0.0190,0.0205,default,1,0.03,0.6888,54681.59,0.15,0,-6240.495325,-10224.806295
3,BTC-27DEC24-110000-C,-28.9,BTC,2024-12-27,110000.0,call,0.405479,56650.4200,1722896484201,70.00,BTC-27DEC24-110000-C,54690.77,18.00400,-14.98803,61.40761,0.00001,-1.481796e+05,68.95,71.25,0.0160,0.0185,default,1,0.03,0.7000,54681.59,0.15,0,-17970.433026,-29512.524942
4,BTC-27DEC24-115000-C,-1.7,BTC,2024-12-27,115000.0,call,0.405479,56650.4200,1722896484201,71.24,BTC-27DEC24-115000-C,54690.77,15.91767,-13.90918,55.99892,0.00001,-7.716119e+03,70.19,71.72,0.0140,0.0155,default,1,0.03,0.7124,54681.59,0.15,0,-938.562415,-1543.943460
5,BTC-27DEC24-120000-C,95.0,BTC,2024-12-27,120000.0,call,0.405479,56651.1200,1722896485638,72.26,BTC-27DEC24-120000-C,54690.77,14.05379,-12.82662,50.90851,0.00001,3.807897e+05,71.56,72.68,0.0125,0.0135,default,1,0.03,0.7226,54681.59,0.15,0,46478.495249,76600.904866
6,BTC-27DEC24-43000-P,31.4,BTC,2024-12-27,43000.0,put,0.405479,56651.1200,1722896484241,60.14,BTC-27DEC24-43000-P,54690.77,-49.59689,-19.45609,92.78827,0.00001,-3.390047e+05,59.51,61.04,0.0430,0.0455,default,1,0.03,0.6014,54681.59,0.15,0,-28305.374918,-38910.784750
7,BTC-27DEC24-44000-P,20.6,BTC,2024-12-27,44000.0,put,0.405479,56651.1200,1722896486881,59.99,BTC-27DEC24-44000-P,54690.77,-54.30537,-20.46746,97.85803,0.00001,-2.413304e+05,59.33,60.78,0.0480,0.0505,default,1,0.03,0.5999,54681.59,0.15,0,-20252.780805,-27902.519103
8,BTC-27DEC24-45000-P,14.0,BTC,2024-12-27,45000.0,put,0.405479,56651.4400,1722896486980,59.87,BTC-27DEC24-45000-P,54690.77,-59.24121,-21.45319,102.76855,0.00001,-1.772816e+05,59.23,60.61,0.0535,0.0560,default,1,0.03,0.5987,54681.59,0.15,0,-14953.396630,-20647.116599
9,BTC-27DEC24-47000-P,-9.0,BTC,2024-12-27,47000.0,put,0.405479,56652.7900,1722896487328,59.70,BTC-27DEC24-47000-P,54690.77,-69.70734,-23.30240,111.94233,0.00001,1.316637e+05,59.24,60.51,0.0660,0.0685,default,1,0.03,0.5970,54681.59,0.15,0,11217.248236,15556.634567
